This notebook holds the solutions for the given objectives:  
⚡Which region has the highest profit, and which country within that region has the highest profit?  
⚡What is the highest number of orders placed within a 72-hour period?  
⚡Which month has the highest average number of units sold?  
⚡On analyzing the financial performance of placed orders. Identify the lowest and highest margin percentages from this analysis?  
⚡On identifying key sales trends d etermine the top 3 highest-value orders for a specific item type from the dataset?  
⚡Which type of item is most frequently sold on weekends in the Sub-Saharan Africa region?  
⚡Compute year-over-year growth for total revenue.  

Here is the data dictionary for the same:  
__Region:__ The broad geographical region of the sale (e.g., “Europe”, “Sub-Saharan Africa”).  
__Country:__ The specific country within the region where the sale occurred.  
__Item Type__: The category of the product sold (e.g., “Baby Food”, “Office Supplies”).  
__Sales Channel__: The method of sale, either “Online” or “Offline”.  
__Order Priority__: The priority level assigned to the order (e.g., “H”, “C”, “L”).  
__Order Date__: The date the order was placed.  
__Order ID__: A unique identifier for the order.  
__Ship Date__: The date the order was shipped.  
__Units Sold__: The quantity of the product sold.  
__Unit Price__: The selling price per unit of the product.  
__Unit Cost__: The cost price per unit of the product.  
__Total Revenue__: The total revenue generated from the sale (Units Sold * Unit Price).  
__Total Cost__: The total cost incurred for the sale (Units Sold * Unit Cost).  
__Total Profit__: The total profit made from the sale (Total Revenue — Total Cost).  

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .appName("ShippingDataAnalysis") \
        .getOrCreate()
        
df = spark.read.csv("AmazonSalesData.csv")

TypeError: 'Builder' object is not callable

---

Which region has the highest profit, and which country within that region has the highest profit?